This notebook is just to try the new stable audio model to generate sound samples.
Before you start, follow the next steps:


1.   Create an account in Hugging Face and
2.   Request access to the model [here](https://huggingface.co/stabilityai/stable-audio-open-1.0).
3. Instal dependecies with the following block and restart the notebook.
4. Check that you are using a GPU, CPU generation can be really slow.

In [ ]:
!pip install einops
!pip install stable-audio-tools
!pip install huggingface_hub

In [1]:
#Insert your generated token here.
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# Set up text and timing conditioning
conditioning = [{
    "prompt": "a jar full of coins being agitated",
    "seconds_start": 0,
    "seconds_total": 5
}]


In [ ]:
import torch
import torchaudio
from einops import rearrange
from stable_audio_tools import get_pretrained_model
from stable_audio_tools.inference.generation import generate_diffusion_cond

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device, "This is the device")

# Download model
model, model_config = get_pretrained_model("stabilityai/stable-audio-open-1.0")
sample_rate = model_config["sample_rate"]
sample_size = model_config["sample_size"]

model = model.to(device)

# Generate stereo audio
output = generate_diffusion_cond(
    model,
    steps=100,
    cfg_scale=7,
    conditioning=conditioning,
    sample_size=sample_size,
    sigma_min=0.3,
    sigma_max=500,
    sampler_type="dpmpp-3m-sde",
    device=device
)

# Rearrange audio batch to a single sequence
output = rearrange(output, "b d n -> d (b n)")

# Peak normalize, clip, convert to int16, and save to file
output = output.to(torch.float32).div(torch.max(torch.abs(output))).clamp(-1, 1).mul(32767).to(torch.int16).cpu()
torchaudio.save("coins.wav", output, sample_rate)